In [98]:
!pip install webdriver-manager
!pip install pandas requests beautifulsoup4 yfinance tqdm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [100]:
# import thư viện
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import time
import random
from selenium import webdriver
import numpy as np
from time import sleep
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import mysql.connector
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import io
from io import StringIO
from concurrent.futures import ThreadPoolExecutor, as_completed
import re
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

In [101]:
## HAM 1: CRAWL 

API_URL = 'https://inav.ice.com/api/1/tse/iopv/table?type=etf&language=en'
HEADERS = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://inav.ice.com'}

def get_etf_code_time():
    try:
        r = requests.get(API_URL, headers=HEADERS, timeout=10)
        r.raise_for_status()
        rows = r.json().get("rows", [])
        
        data = []
        for row in rows:
            code = row.get("code")
            time = row.get("time")
            volume = row.get("volume")
            if code and time:
                data.append({"code": code, "time": time, "volume": volume})
        
        return data
    except requests.RequestException as e:
        print(f"⚠️ Lỗi khi lấy dữ liệu ETF từ API: {e}")
        return []

data = get_etf_code_time()
df_1 = pd.DataFrame(data)


# --- Cấu hình ---
MAX_WORKERS = 12

# --- HÀM TỔNG HỢP: LẤY OHLC (ROBUST) VÀ CURRENCY CÙNG LÚC ---
def fetch_data_yf(etf_code):
    """
    Tải OHLC (Previous Close) và Currency trong một lần gọi API, xử lý lỗi thiếu dữ liệu lịch sử.
    """
    ticker = f"{etf_code}.T"
    
    # Khởi tạo kết quả mặc định là None/Null
    result = {"code": etf_code, "open": None, "high": None, "low": None, "close": None, "currency": None}
    
    try:
        stock = yf.Ticker(ticker)
        
        # 1. Lấy Currency từ info
        info = stock.info
        result["currency"] = info.get("currency")
        
        # 2. Lấy OHLC (dữ liệu 2 ngày để có Previous Close)
        hist = stock.history(period="2d", interval="1d")
        
        if len(hist) >= 2:
            # Trường hợp lý tưởng: Có đủ 2 ngày. Lấy Previous Close
            today_data = hist.iloc[-1]
            previous_day_data = hist.iloc[-2]

            result["open"] = today_data["Open"]
            result["high"] = today_data["High"]
            result["low"] = today_data["Low"]
            result["close"] = previous_day_data["Close"] # Previous Close
            
        elif len(hist) == 1:
            # Trường hợp chỉ có 1 ngày (Lấy Last Available Close)
            last_available_data = hist.iloc[-1]
            result["open"] = last_available_data["Open"]
            result["high"] = last_available_data["High"]
            result["low"] = last_available_data["Low"]
            result["close"] = last_available_data["Close"] # Last Available Close

    except Exception:
        # Bỏ qua lỗi và trả về None
        pass
        
    return result

# --- HÀM CHÍNH: TẠO DF_PRICE_FINAL ---
def create_df_price_final(df_1):
    """
    Thực hiện tải song song (OHLC & Currency) và hợp nhất vào df_1.
    """
    
    # Giả định df_1 có cột: code, time, volume
    codes_list = df_1['code'].tolist()
    total_codes = len(codes_list)
    
    print(f"\n1. Bắt đầu tải song song OHLC/Currency cho TẤT CẢ {total_codes} codes...")

    final_results = []
    
    # Tải song song
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futures = {ex.submit(fetch_data_yf, code): code for code in codes_list}
        
        for i, future in tqdm(enumerate(as_completed(futures), 1), total=total_codes, desc="Tải Dữ liệu YF"):
            final_results.append(future.result())

    print("\n--- Hoàn tất tải dữ liệu ---")

    # Tạo DataFrame tạm thời
    df_new_data = pd.DataFrame(final_results)
    
    # Hợp nhất df_new_data vào df_1 gốc
    df_price_final = df_1.merge(df_new_data, on='code', how='left')

    # Đảm bảo thứ tự 8 cột cuối cùng theo yêu cầu
    final_columns = ['code', 'time', 'volume', 'open', 'high', 'low', 'close', 'currency']
    df_price_final = df_price_final[final_columns] 

    print("\n2. Đã tạo DataFrame **df_price_final** với 8 cột.")
    print("\n--- df_price_final (Kết quả đầu tiên) ---")
    print(df_price_final.head().to_markdown(index=False))
    
    return df_price_final

# --- THỰC THI ---
# Chạy hàm này sau khi df_1 đã được định nghĩa:

df_price_final = create_df_price_final(df_1)


1. Bắt đầu tải song song OHLC/Currency cho TẤT CẢ 386 codes...


Tải Dữ liệu YF: 100%|██████████████████████████████████████████████████████████████| 386/386 [00:20<00:00, 18.60it/s]


--- Hoàn tất tải dữ liệu ---

2. Đã tạo DataFrame **df_price_final** với 8 cột.

--- df_price_final (Kết quả đầu tiên) ---
|   code | time        | volume   |   open |   high |   low |   close | currency   |
|-------:|:------------|:---------|-------:|-------:|------:|--------:|:-----------|
|   1305 | 12/05 11:30 | 13,720   |   3569 |   3570 |  3547 |    3598 | JPY        |
|   1306 | 12/05 11:30 | 906,970  |   3528 |   3535 |  3511 |    3566 | JPY        |
|   1308 | 12/05 11:30 | 57,536   |   3484 |   3491 |  3469 |    3516 | JPY        |
|   1309 | 12/05 11:30 | 21       |  52280 |  52280 | 52070 |   52280 | JPY        |
|   1311 | 12/05 11:30 | 6,150    |   1769 |   1775 |  1761 |    1792 | JPY        |


In [102]:
skip_col = ['code', 'time', 'currency','volume']

# Convert tất cả cột không nằm trong skip_col
for col in df_price_final.columns:
    if col not in skip_col:
        df_price_final[col] = pd.to_numeric(df_price_final[col], errors='coerce')
# Convert các cột cần chuyển sang int
cols_to_convert = ['open', 'high', 'low', 'close']

for col in cols_to_convert:
    df_price_final[col] = pd.to_numeric(df_price_final[col], errors='coerce')
    df_price_final[col] = df_price_final[col].fillna(0)
    df_price_final[col] = df_price_final[col].astype(int)

df_price_final['volume'] = df_price_final['volume'].astype(str).str.replace(',','')
df_price_final['volume'] = df_price_final['volume'].astype(int)

# Năm cần gán
CURRENT_YEAR = 2025 

# 1. Thêm năm 2025 vào chuỗi thời gian gốc
# Ví dụ: "12/02 11:27" -> "12/02 11:27/2025"
df_price_final['time_with_year'] = df_price_final['time'].astype(str) + '/' + str(CURRENT_YEAR)

# 2. Chuyển đổi chuỗi mới sang đối tượng datetime
# Định dạng mới phải phù hợp: %m/%d %H:%M/%Y
df_price_final['date'] = pd.to_datetime(
    df_price_final['time_with_year'], 
    format='%m/%d %H:%M/%Y', 
    errors='coerce'
)
# 3. Xóa cột trung gian
df_price_final.drop(columns=['time_with_year'], inplace=True)
df_price_final.drop(columns=['time'], inplace=True)
df_price_final[['date','code','open','high','low','close','volume','currency']]

df_price_final.rename(columns={'code':'etf_code'}, inplace=True)







In [103]:
# --- Hàm 3: UPSERT MYSQL


def upsert_to_mysql(df: pd.DataFrame, table_name: str = 'etf_historical_price_train'):
    """
    Thực hiện kết nối và UPSERT (Update or Insert) dữ liệu từ DataFrame vào MySQL.
    
    """
    try:
        # 1. Kết nối MySQL (Sử dụng thông tin của bạn)
        cnx = mysql.connector.connect(
            user='root',
            password='Phunghainam2004!',
            host='127.0.0.1',
            database='train'
        )
        cursor = cnx.cursor()

        # 2. Định nghĩa các cột
        # Bỏ cột 'id' nếu nó là AUTO_INCREMENT. Ở đây, ta dùng tất cả cột trừ 'id' để update.
        cols_to_use = [
            'date', 'etf_code', 'open', 'high', 
            'low', 'close','volume', 'currency'
        ]
        
        # 3. Tạo cú pháp SQL cho UPSERT
        col_str = ", ".join([f"`{c}`" for c in cols_to_use])
        placeholder_str = ", ".join(["%s"] * len(cols_to_use))
        
        # Tạo chuỗi cho phần ON DUPLICATE KEY UPDATE
        # Các cột sẽ được cập nhật nếu 'etf_code' đã tồn tại
        update_str = ", ".join([f"`{c}` = VALUES(`{c}`)" for c in cols_to_use])

        sql = f"""
        INSERT INTO `{table_name}` ({col_str})
        VALUES ({placeholder_str})
        ON DUPLICATE KEY UPDATE
        {update_str}
        """

        # 4. Chuẩn bị dữ liệu và thực thi
        df_to_insert = df[cols_to_use]
        data_to_insert = [
            tuple(None if pd.isna(x) else x for x in row) 
            for row in df_to_insert.itertuples(index=False, name=None)
        ]
        
        # Sử dụng executemany để chèn nhiều dòng cùng lúc (tối ưu hơn loop)
        cursor.executemany(sql, data_to_insert)
        
        # 5. Commit và Đóng kết nối
        cnx.commit()
        print(f"✅ Đã UPSERT {cursor.rowcount} dòng vào bảng `{table_name}` thành công!")

    except mysql.connector.Error as err:
        print(f"❌ Lỗi MySQL: {err}")
    
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx and cnx.is_connected():
            cnx.close()


# --- THỰC THI HÀM 5 (Chạy cell này trong Notebook) ---

# Đảm bảo cột 'id' được loại bỏ nếu bạn không muốn insert giá trị cho nó (AUTO_INCREMENT)
# Nếu bạn muốn dùng cột 'id' đã tạo để làm khóa chính, bạn cần thêm nó vào `cols_to_use`.
# Ở đây ta chỉ dùng các cột có dữ liệu crawl được.
df_for_upsert = df_price_final

upsert_to_mysql(df_for_upsert)

✅ Đã UPSERT 546 dòng vào bảng `etf_historical_price_train` thành công!
